# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [2]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [3]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [4]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try: 
    cluster = Cluster(['127.0.0.1']) #If you have a locally installed Apache Cassandra instance
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [7]:
# Create a Keyspace 

try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS project1b
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
# Set KEYSPACE to the keyspace specified above

try:
    session.set_keyspace('project1b')
except Exception as e:
    print(e)


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [8]:
## Query 1:  Give me the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4

# Drop the table if alredy exists

qu = "DROP TABLE IF Exists musicapp_songs;"
try:
    session.execute(qu)
except Exception as e:
    print(e)
    
#Create table based ont the where clause for the primary key columns

query = "CREATE TABLE IF NOT EXISTS musicapp_songs"
query = query + "(sessionid int,iteminsession int,artist text,songlength float, songtitle text,PRIMARY KEY (sessionid,iteminsession))"
try:
    session.execute(query)
except Exception as e:
    print(e)


In [12]:

file = 'event_datafile_new.csv'
#open the file and insert data

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    next(csvreader)
    for line in csvreader:        
        ## INSERT statements into the `query` variable
        query = "INSERT INTO musicapp_songs(sessionid,iteminsession,artist,songlength, songtitle)"
        query = query + " VALUES (%s,%s,%s,%s,%s)"
        
        ## Assign which column element should be assigned for each column in the INSERT statement.
        #print(query, ((str(line[0]), int(line[3]),float(line[5]), int(line[8]),str(line[9]))))
        session.execute(query, (int(line[8]),int(line[3]),str(line[0]), float(line[5]) ,str(line[9])))
        
       

#### Do a SELECT to verify that the data have been inserted into each table



In [13]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table

query = "select artist,songtitle,songlength from musicapp_songs where sessionid =338 and itemInSession  = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.songtitle, row.songlength)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [25]:
## TO-DO: Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
qu = "DROP TABLE IF Exists musicapp_artist;"
try:
    session.execute(qu)
except Exception as e:
    print(e)
    
# use the itemsession, first name and last name as clustered key such that the data is ordered by them
query = "CREATE TABLE IF NOT EXISTS musicapp_artist"
query = query + "( userid int,sessionid int,iteminsession int,artist text,\
                    songlength float, songtitle text, firstname text, lastname text,\
                    PRIMARY KEY ((userid,sessionid),iteminsession))"
try:
    session.execute(query)
except Exception as e:
    print(e)
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    next(csvreader)
    for line in csvreader:
        
        ## INSERT statements into the `query` variable
        query = "INSERT INTO musicapp_artist(sessionid,userid,iteminsession,artist,songlength, songtitle,firstname,lastname)"
        query = query + " VALUES (%s,%s,%s,%s,%s,%s,%s,%s)"
        
        ## Assign which column element should be assigned for each column in the INSERT statement.
        #print(query, ((str(line[0]), int(line[3]),float(line[5]), int(line[8]),str(line[9]))))        
        session.execute(query, (int(line[10]),int(line[8]),int(line[3]),str(line[0]), float(line[5]), str(line[9]),str(line[1]),str(line[4])))
        
# Quey the inserted data. based on the 2 primary keys         
query = "select artist,songtitle,firstname,lastname,iteminsession from musicapp_artist where userid = 10 and sessionid = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.songtitle, row.firstname,row.lastname,row.iteminsession)
        
                  



Down To The Bone Keep On Keepin' On Sylvie Cruz 0
Three Drives Greece 2000 Sylvie Cruz 1
Sebastien Tellier Kilometer Sylvie Cruz 2
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz 3


In [22]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

qu = "DROP TABLE IF Exists musicapp_usersongs;"
try:
    session.execute(qu)
except Exception as e:
    print(e)
    

#Create table with composite key, songtitle is first column and rest all uniquely identifiable columns are 
#composite/ cluster columns. Included sessionID on the primary key in case the same user logs in multiple times and
#hears to multiple songs

query = "CREATE TABLE IF NOT EXISTS musicapp_usersongs"
query = query + "(songtitle text, userid int,sessionid int,  firstname text, \
                    lastname text,PRIMARY KEY (songtitle,userid,sessionid))"
try:
    session.execute(query)
except Exception as e:
    print(e)
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    next(csvreader)
    for line in csvreader:
        
        ## INSERT statements into the `query` variable
        query = "INSERT INTO musicapp_usersongs( songtitle,userid,sessionid,firstname,lastname)"
        query = query + " VALUES (%s,%s,%s,%s,%s)"
        
        ## Assign which column element should be assigned for each column in the INSERT statement.
        #print(query, (int(line[8]),int(line[10]),str(line[9]),str(line[1]),str(line[4])))    
        session.execute(query, (str(line[9]),int(line[10]),int(line[8]),str(line[1]),str(line[4])))
        
query = "select songtitle,firstname,lastname from musicapp_usersongs where songtitle='All Hands Against His Own'"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row)
    print (row.songtitle, row.firstname,row.lastname)
        
                  


                    

Row(songtitle='All Hands Against His Own', userid=29, sessionid=559, firstname='Jacqueline', lastname='Lynch')
All Hands Against His Own Jacqueline Lynch
Row(songtitle='All Hands Against His Own', userid=80, sessionid=611, firstname='Tegan', lastname='Levine')
All Hands Against His Own Tegan Levine
Row(songtitle='All Hands Against His Own', userid=95, sessionid=152, firstname='Sara', lastname='Johnson')
All Hands Against His Own Sara Johnson


### Drop the tables before closing out the sessions

In [23]:
## TO-DO: Drop the table before closing out the sessions

qu = "DROP TABLE IF Exists musicapp;"
try:
    session.execute(qu)
except Exception as e:
    print(e)

    qu = "DROP TABLE IF Exists musicapp1;"
try:
    session.execute(qu)
except Exception as e:
    print(e)

qu = "DROP TABLE IF Exists musicapp2;"
try:
    session.execute(qu)
except Exception as e:
    print(e)


### Close the session and cluster connection¶

In [24]:
session.shutdown()
cluster.shutdown()